In [ ]:
# IGNORE THIS CELL WHICH CUSTOMIZES LAYOUT AND STYLING OF THE NOTEBOOK !
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings = lambda *a, **kw: None

# Exercise 0.5 - pandas
prepared by M.Hauser

[pandas](https://pandas.pydata.org/) is a "Python package providing fast, flexible, and expressive data structures designed to make working with 'relational' or 'labeled' data both easy and intuitive. It aims to be the fundamental high-level building block for doing practical, real world data analysis in Python. Additionally, it has the broader goal of becoming the most powerful and flexible open source data analysis / manipulation tool available in any language."

Thus, pandas provides R-like data structures, e.g. a `DataFrame`, and associated methods. Pandas's core strength is the handling of 2D data and some of its functionality is very handy.

A more comprehensive introduction is given in [10 Minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html).

In [ ]:
# pandas is conventionally abbreviated as `pd`
import numpy as np
import pandas as pd

## Data types

Pandas offers several data types - the most important are the Series and DataFrame:

### Series

Series are 1-dimensional, and each element is labeled, in our example with 'a', 'b', .... The labels are collectively referred to as the index. 


In [ ]:
data = np.random.randn(5)
s = pd.Series(data, index=["a", "b", "c", "d", "e"])

s

### DataFrame

DataFrame is the most common data type of pandas. It is 2-dimensional, has an index and (potentially) several columns.

In [ ]:
time = pd.date_range("2013-01-28", periods=6)

data = np.random.randn(6, 4)

df = pd.DataFrame(data, index=time, columns=list("ABCD"))

df

## Selection

There are various ways to select data, see the [introduction](https://pandas.pydata.org/pandas-docs/stable/10min.html#selection).

#### select the column named 'A' - as `Series`

In [ ]:
df["A"]

#### select the column named 'A' - as `DataFrame`

> note the double brackets!

In [ ]:
df[["A"]]

#### select two columns

In [ ]:
df[["A", "B"]]

### with `df.loc[...]` you can select rows

#### select a single day - as `Series`

In [ ]:
df.loc["2013-01-30"]

#### select a single day - as `DataFrame`

In [ ]:
df.loc["2013-01-30":"2013-01-30"]

#### select all days in a month

In [ ]:
df.loc["2013-01"]

### with `df.iloc[...]` you can select by position

#### select the first three elements

In [ ]:
df.iloc[:3]

#### item assignment

In [ ]:
# item assignment can also be done

df.iloc[3, 1] = np.NaN

df.loc["2013-02", "D"] = np.NaN

df

## Computation

#### computations are usually done along the index axis

In [ ]:
df.mean()

#### unless you explicitly set the axis

In [ ]:
df.mean(axis=1)

Missing values (`NaN`) are generally left out. Unless explicitly included:

In [ ]:
df.mean(skipna=False)

### Exercises

 * What happens if we add a column named 'E', and only assign a value for the first day? (Hint: `df.loc['2013-01-28', 'E'] = 5`)
 * Add a new column named 'F' where all values are 2.
 * From what you know, calculate the standard deviation of the values in February, for all columns.

### Solution

In [ ]:
# all the other rows are NaN

df.loc["2013-01-28", "E"] = 5
print("Added row 'E'")
print(df)
print()

df.loc[:, "F"] = 2
print("Added row 'F'")
print(df)

print()
df.loc["2013-02", :].std()

## Reading comma separated values (csv) from a file

For the rest of the exercise we use measurements of the atmospheric CO<sub>2</sub> concentration from Mauna Loa, Hawaii (Keeling & Whorf 2004), see how the [data was prepared](./../data/prepare_CO2_mauna_loa.ipynb).

The CO$_2$ data is saved in a .csv file. Let's look at the first lines of the file:

In [ ]:
fN = "./../data/co2.csv"

with open(fN) as fid:
    for i in range(10):
        print(fid.readline(), end="")

pandas has a powerful function to read from .csv files, `pd.read_csv`. Although the defaults are quite good, it usually requires some tweaking... (I always need to check the documentation...). 

For our file we need to pass at least the filename (`fN`) and the number of the header rows, (which gives the name for the column, here 'co2'). Because python is zero-based this is line number 6 - 1 = 5, thus, we pass `header=5`.

In [ ]:
co2_prelim = pd.read_csv(fN, header=5)

co2_prelim.head(3)

let's check how the date format is saved

In [ ]:
print('The first date "{:s}"'.format(co2_prelim.iloc[0, 0]))
print("The type of the date", type(co2_prelim.iloc[0, 0]))

This is not entirely what we want, yet.

 * We want the labels to be the (time) axis, and not numbers. Therefore we have to choose `index_col=0`, which uses the first column as index.
 * We also want the time axis to be a pandas [`DateTimeIndex`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.html), and not a string. Thus, we use `parse_dates=True`.
 
See `pd.read_csv?` for details.

In [ ]:
co2 = pd.read_csv(fN, index_col=0, parse_dates=True, header=5)

co2.head()

In [ ]:
co2.index

## Time Series

pandas has very powerful [time series functionality](https://pandas.pydata.org/pandas-docs/stable/timeseries.html#time-series). We already saw the selection of months by using `df.loc['2013-02', :]`. `resample` is another, very helpful function. We can use it to convert the weekly CO$_2$ time series to a monthly time series:

In [ ]:
co2_monthly = co2.resample("M").mean()

co2_monthly.head()

Another time series functionality I often use is [`date_range`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.date_range.html). It can be used to construct `DatetimeIndex` with a fixed frequency.

#### every day of the year 2000

In [ ]:
pd.date_range("2000-01-01", "2000-12-31")

In [ ]:
# every month of the year 2000
pd.date_range("2000-01-01", "2000-12-31", freq="M")

In [ ]:
# every second month of the year 2000
pd.date_range("2000-01-01", "2000-12-31", freq="2M")

**Note:** Different options available for the freq parameter (also called "offset aliases") can be found at this [link](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)

### Exercises

* Calculate annual mean CO$_2$ using `resample`. (Hint: [offset aliases](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases))
* Create a `DatetimeIndex` from 01.05.2000 to 31.05.2000 with a half-hourly frequency.
* Calculate the standard deviation of `df` for each month again, this time using `resample`.

### Solution

In [ ]:
co2.resample("A").mean().head()

In [ ]:
pd.date_range("2000-05-01", "2000-05-31", freq="30min")

In [ ]:
df.resample("M").std()